In [ ]:
import requests
from datetime import date, timedelta
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

## Recursos:
- [IP get info](https://ipinfo.io/)
- [NASA API](https://api.nasa.gov/)
- [How to Create a 3D Model of the Solar System Using Plotly in Python](https://python.plainenglish.io/how-to-create-a-3d-model-of-the-solar-system-with-plotly-in-python-2a74e672b771)


## Identificación de la ubicación:
La obtención de la ubicación se logra mediante el uso de la dirección IP a través de la API de ipinfo.io.

In [ ]:
# Realiza una solicitud a ipinfo.io para obtener la información de ubicación basada en la IP
response = requests.get('https://ipinfo.io')

# Analiza la respuesta JSON
data = response.json()

# Extrae la latitud y longitud si están disponibles
if 'loc' in data:
    latitud, longitud = data['loc'].split(',')
    print(f'Latitud: {latitud}')
    print(f'Longitud: {longitud}')
else:
    print('No se pudo obtener la información de ubicación.')

""" Recordar que la locacion identificada corresponde de la maquina virtual sobre la que corremos el script """

Latitud: 53.2192
Longitud: 6.5667


' Recordar que la locacion identificada corresponde de la maquina virtual sobre la que corremos el script '

In [ ]:
# Crear un DataFrame con los datos y agrego mi locacion real
data = {'location': ['Colab','House'],'Latitud': [latitud,-34.0981], 'Longitud': [longitud,-59.0286]}
df = pd.DataFrame(data)

In [ ]:
# Crear una figura de Plotly Express para el mapa
fig = px.scatter_geo(df, lat='Latitud', lon='Longitud', scope='world')

# Personalizar el diseño del mapa
fig.update_geos(projection_type='orthographic')
fig.add_trace(px.scatter_geo(df, lat='Latitud', lon='Longitud', text='location').data[0])
fig.update_layout(title='Ubicaciones', geo=dict(showland=True))
fig.show()

## NASA API

In [ ]:
import ephem
def calcular_fase_lunar(latitud, longitud, fecha_inicio):
    # Crear un observador para la ubicación y fecha especificadas.
    observer = ephem.Observer()
    observer.lat = latitud
    observer.long = longitud
    observer.date = fecha_inicio

    # Calcular la fase lunar.
    moon = ephem.Moon()
    moon.compute(observer)

    # Obtener la fase lunar como un valor decimal y convertirla a porcentaje.
    fase_porcentaje = (moon.moon_phase / 29.53) * 100

    return fase_porcentaje

In [ ]:
for index, row in df.iterrows():
    ubicacion = row['location']
    latitud = row['Latitud']
    longitud = row['Longitud']
    fecha_inicio = date.today()
    fase_porcentaje = calcular_fase_lunar(latitud, longitud, fecha_inicio)

    # Imprimir la ubicación y la fase lunar en porcentaje.
    print(f"Ubicación: {ubicacion}, Fase Lunar (en %): {fase_porcentaje:.2f}%")

Ubicación: Colab, Fase Lunar (en %): 0.29%
Ubicación: House, Fase Lunar (en %): 0.29%


In [ ]:
# Configura tu clave de API de la NASA
api_key = 'PSNQRV93quuyth4x59j2YFK7D0PXJXHQjxnizShQ'

In [ ]:
# Obtener la fecha de inicio (1 de enero de 2023)
fecha_inicio = date(2023, 1, 1)

# Obtener la fecha de fin (31 de diciembre de 2023)
fecha_fin = date(2023, 12, 31)

# Crear un DataFrame vacío para almacenar los eventos astronómicos
eventos_df = []

# Consultar la API APOD para cada día del año 2023
fecha = fecha_inicio
while fecha <= fecha_fin:
    fecha_str = fecha.strftime('%Y-%m-%d')
    url = f'https://api.nasa.gov/planetary/apod?date={fecha_str}&api_key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        eventos_df.append({
            'Fecha': fecha_str,
            'Título': data.get('title', ''),
            'Explicación': data.get('explanation', ''),
            'URL': data.get('url', ''),
            'Créditos': data.get('copyright', ''),
            'HdURL': data.get('hdurl', ''),
            'MediaTipo': data.get('media_type', ''),
        })
    fecha += timedelta(days=1)



In [ ]:
eventos_df = pd.DataFrame(eventos_df)

In [ ]:
palabras_clave = {
    'Cometa': ['Comet'],
    'Estacion': ['Space Station', 'ISS', 'Orbit', 'Space Station', 'Low Earth Orbit'],
    'Nebulosa': ['Nebula', 'Nebular', 'Nebulosity', 'Galaxy', 'Stellar Cloud'],
    'Luna': ['Moon', 'Lunar'],
    'Planeta': ['Planet', 'Planetary', 'Mars', 'Venus', 'Jupiter', 'Saturn', 'Neptune', 'Uranus', 'Mercury'],
    'Agujero Negro': ['Black Hole'],
    'Asterismo': ['Asterism'],
    'Constelación': ['Constellation'],
    'Supernova': ['Supernova'],
    'Estrella': ['Star', 'Stellar'],
    'Meteorito': ['Meteor', 'Meteorite'],
    'Eclipse': ['Eclipse', 'Solar Eclipse', 'Lunar Eclipse'],
    'Galaxia': ['Galaxy', 'Spiral Galaxy', 'Elliptical Galaxy'],
    'Planeta Enano': ['Dwarf Planet'],
    'Estación Espacial': ['Space Station', 'International Space Station', 'ISS'],
    'Satélite Natural': ['Moon', 'Lunar'],
    'Cúmulo Estelar': ['Star Cluster'],
    'Evento Celestial': ['Celestial Event'],
    'Otro': [],  # Si no se encuentra ninguna coincidencia, se clasifica como "Otro"
}


In [ ]:
# Función para asignar una categoría a un evento basada en las palabras clave
def asignar_categoria(descripcion):
    for categoria, palabras in palabras_clave.items():
        for palabra in palabras:
            if palabra.lower() in descripcion.lower():
                return categoria
    return 'Otro'  # Si no se encuentra ninguna coincidencia, asigna la categoría "Otro"

In [ ]:
# Definir una función para asignar subcategorías a los eventos de eclipse
def asignar_subcategoria(descripcion):
    if 'Solar Eclipse' in descripcion:
        return 'Solar'
    elif 'Lunar Eclipse' in descripcion:
        return 'Lunar'
    else:
        return 'N/A'  # Si no se encuentra ninguna coincidencia, se asigna "N/A"

In [ ]:
# Aplicar la función para asignar categorías a cada evento
eventos_df['Categoria'] = eventos_df['Explicación'].apply(asignar_categoria)

# Aplicar la función para asignar subcategorías a los eventos de eclipse
eventos_df['Subcategoria'] = eventos_df['Explicación'].apply(asignar_subcategoria)

# Convertir la columna de Fecha a tipo de dato datetime
eventos_df['Fecha'] = pd.to_datetime(eventos_df['Fecha'])

# Agregar una columna de Mes para agrupar por mes
eventos_df['Mes'] = eventos_df['Fecha'].dt.strftime('%B')

In [ ]:
eventos_df.tail(5)

,Fecha,Título,Explicación,URL,Créditos,HdURL,MediaTipo,Categoria,Subcategoria,Mes
249,2023-09-07,The Large Cloud of Magellan,The 16th century Portuguese navigator Ferdinan...,https://apod.nasa.gov/apod/image/2309/TheLarge...,Chris Willocks,https://apod.nasa.gov/apod/image/2309/TheLarge...,image,Nebulosa,N/A,September
250,2023-09-08,Star Factory Messier 17,"Sculpted by stellar winds and radiation, the s...",https://apod.nasa.gov/apod/image/2309/OmegaNeb...,Kim Quick,https://apod.nasa.gov/apod/image/2309/OmegaNeb...,image,Nebulosa,N/A,September
251,2023-09-09,Comet Nishimura Grows,"Comet Nishimura is growing. More precisely, th...",https://apod.nasa.gov/apod/image/2309/Nishimur...,\nPeter Kennett\n,https://apod.nasa.gov/apod/image/2309/Nishimur...,image,Cometa,N/A,September
252,2023-09-10,An Annular Solar Eclipse over New Mexico,"What is this person doing? In 2012, an annula...",https://apod.nasa.gov/apod/image/2309/AnnularE...,\nColleen Pinski\n,https://apod.nasa.gov/apod/image/2309/AnnularE...,image,Luna,N/A,September
253,2023-09-11,Beautiful Comet Nishimura,This scene would be beautiful even without the...,https://apod.nasa.gov/apod/image/2309/BeautNis...,\nPetr Horálek /\nInstitute of Physics in Opava\n,https://apod.nasa.gov/apod/image/2309/BeautNis...,image,Cometa,N/A,September


In [ ]:
import calendar

# Definir el orden personalizado de los meses
orden_meses = [calendar.month_name[i] for i in range(1, 13)]

# Aplicar el orden personalizado a la columna 'Mes' en el DataFrame
eventos_df['Mes'] = pd.Categorical(eventos_df['Mes'], categories=orden_meses, ordered=True)

In [ ]:
# Agrupar los eventos por Mes y Categoría y contar cuántos hay en cada categoría para cada mes
conteo_eventos = eventos_df.groupby(['Mes', 'Categoria','Subcategoria']).size().reset_index(name='Conteo')

# Crear un gráfico de barras apiladas
fig = px.bar(conteo_eventos, x='Mes', y='Conteo', color='Categoria', title='Eventos Astronómicos por Mes y Categoría en 2023')

# Mostrar el gráfico
fig.show()

In [ ]:
# Crear un gráfico de barras apiladas con subcategoría
fig = px.bar(conteo_eventos.loc[conteo_eventos['Subcategoria'] != "N/A"], x='Subcategoria', y='Conteo', color='Mes',
             title='Distribución de Eventos Astronómicos por Categoría y Subcategoría en 2023',
             labels={'Categoria': 'Categoría', 'Conteo': 'Número de Eventos'})

# Mostrar el gráfico
fig.show()

In [ ]:
fig = px.sunburst(conteo_eventos, path=['Mes', 'Categoria', 'Subcategoria'], values='Conteo')
fig.show()